In [1]:
import os
from typing import List

import numpy as np
import pandas as pd
import seaborn as sns
from comet_ml import Experiment
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import tensorflow_hub as hub

In [2]:
train_df = pd.read_csv('../data/tweets_50_shuffled_train.csv')
test_df = pd.read_csv('../data/tweets_50_shuffled_test.csv')
X_train = train_df['text_tokenized']
y_train = train_df['label']
X_test = test_df['text_tokenized']
y_test = test_df['label']

class_names = y_train.unique()

In [3]:
batch_size = 128
tfhub_embedding_models = ["https://tfhub.dev/google/nnlm-en-dim50/2", "https://tfhub.dev/google/nnlm-en-dim128-with-normalization/2"]
activation = 'relu'
kernel_initializer = 'he_normal'
l2_lambda = 1e-3
dropout=0.5
filters = 64
kernel_sizes = [1, 2, 3]
n_convs_parallel = len(kernel_sizes)
padding='same'
pool_size = 2
strides = 1
epochs = 200
optimizer = 'nadam'

for embed_model in tfhub_embedding_models:
    def make_model():
        regularizers = keras.regularizers.l2(l2=l2_lambda)
        model = keras.Sequential([
            hub.KerasLayer(embed_model, dtype=tf.string, input_shape=[],),
            keras.layers.Dense(128, activation=activation, kernel_initializer=kernel_initializer, kernel_regularizer=regularizers),
            keras.layers.Dense(len(class_names), activation="softmax")
        ])
        model.summary()

        model.compile(
            loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["acc"]
        )
        return model

    project_name = 'nlp_split'
    experiment = Experiment(
        project_name=project_name,
        auto_param_logging=True,
        # auto_histogram_weight_logging=True,
        auto_histogram_gradient_logging=True,
        auto_histogram_activation_logging=True,
        api_key="HeH9EtfDC2KUlCOjeQaU1CuOM",
        workspace="henrystoll",
    )
    params = {
        'batch_size': batch_size,
        'bert_model_name': embed_model,
        'filters': filters,
        'kernel_sizes': kernel_sizes,
        'pool_size': pool_size,
        'padding': padding,
        'strides': strides,
        'n_convs_parallel': n_convs_parallel,
        'activation': activation,
        'kernel_initializer': kernel_initializer,
        'l2_lambda': l2_lambda,
        'dropout': dropout,
        'optimizer': optimizer,
        'epochs': epochs,
    }

    experiment.log_parameters(params)

    model = make_model()
    # keras.utils.plot_model(model, "model.png", show_shapes=True)
    # experiment.log_asset("model.png")

    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_acc',
        patience=30,
        verbose=1,
        restore_best_weights=True)

    with experiment.train():
        history = model.fit(X_train, y_train,
                            batch_size=batch_size,
                            epochs=epochs,
                            validation_data=(X_test, y_test),
                            verbose=1,
                            callbacks=[early_stopping])

    with experiment.test():
        loss, accuracy = model.evaluate(X_test, y_test)
        print('acc : {:.3f}'.format(accuracy))
        metrics = {
            'loss': loss,
            'accuracy': accuracy
        }
        experiment.log_metrics(metrics)

    y_predicted = model.predict(X_test)
    y_predicted = y_predicted.argmax(axis=1)
    experiment.log_confusion_matrix(y_test.to_numpy(), y_predicted)

    experiment.end()

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/henrystoll/nlp-split/37267a1079794d92ac6e338015782dee

COMET INFO: Ignoring automatic log_parameter('verbose') because 'keras:verbose' is in COMET_LOGGING_PARAMETERS_IGNORE


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 50)                48190600  
_________________________________________________________________
dense (Dense)                (None, 128)               6528      
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 387       
Total params: 48,197,515
Trainable params: 6,915
Non-trainable params: 48,190,600
_________________________________________________________________
Epoch 1/200
59/59 [==============================] - 7s 87ms/step - loss: 1.2373 - acc: 0.5094 - val_loss: 1.4753 - val_acc: 0.3311
Epoch 2/200
59/59 [==============================] - 4s 72ms/step - loss: 1.0918 - acc: 0.6094 - val_loss: 1.2282 - val_acc: 0.5822
Epoch 3/200
59/59 [==============================] - 4s 71ms/step - loss: 1.0214 - acc: 0.6471 - val_loss:

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/henrystoll/nlp-split/37267a1079794d92ac6e338015782dee
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     test_accuracy             : 0.6415094137191772
COMET INFO:     test_loss                 : 1.0557458400726318
COMET INFO:     train_acc [41]            : (0.509357750415802, 0.8894574046134949)
COMET INFO:     train_batch_acc [246]     : (0.3984375, 0.8991477489471436)
COMET INFO:     train_batch_loss [246]    : (0.44588977098464966, 1.402728796005249)
COMET INFO:     train_epoch_duration [41] : (3.764460638980381, 7.210416896035895)
COMET INFO:     train_loss [41]           : (0.4534895420074463, 1.2372779846191406)
COMET INFO:     train_val_acc [41]        : (0.27403414249420166, 0.6415094137191772)
COMET INFO:     train_val_loss [41] 

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 128)               124642688 
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 124,659,587
Trainable params: 16,899
Non-trainable params: 124,642,688
_________________________________________________________________
Epoch 1/200
59/59 [==============================] - 6s 91ms/step - loss: 1.2059 - acc: 0.5251 - val_loss: 1.4261 - val_acc: 0.3603
Epoch 2/200
59/59 [==============================] - 5s 86ms/step - loss: 1.0372 - acc: 0.6340 - val_loss: 1.9168 - val_acc: 0.1712
Epoch 3/200
59/59 [==============================] - 5s 80ms/step - loss: 0.9525 - acc: 0.6887 - val_

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/henrystoll/nlp-split/4afec5223e3048ff8ac19716e9321202
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     test_accuracy             : 0.6379155516624451
COMET INFO:     test_loss                 : 1.3143929243087769
COMET INFO:     train_acc [99]            : (0.5251110792160034, 0.977783739566803)
COMET INFO:     train_batch_acc [594]     : (0.3203125, 0.9921875)
COMET INFO:     train_batch_loss [594]    : (0.1736261546611786, 2.4800643920898438)
COMET INFO:     train_epoch_duration [99] : (4.593676803982817, 7.574536058993544)
COMET INFO:     train_loss [99]           : (0.2002364546060562, 1.205949068069458)
COMET INFO:     train_val_acc [99]        : (0.171159029006958, 0.6379155516624451)
COMET INFO:     train_val_loss [99]       : (1.1